In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 11 12:01:26 2017

@author: whikwon
"""

import cx_Oracle
import pandas as pd
import os
import QUERY_STATE as qs

dsn_tns = cx_Oracle.makedsn('165.244.121.4', 1521, 'IEPCS1')
db = cx_Oracle.connect('iepcs_view', 'viewdb7388', dsn_tns)

dsn_tns = cx_Oracle.makedsn('165.244.121.64', 3001, 'IEGOS')
db1 = cx_Oracle.connect('iegos_view', 'viewdb7388', dsn_tns)

In [ ]:
#%% 구미 투입실적 정리
os.chdir('//Client/D$/#.Secure Work Folder/2.투입실적/17년 투입 실적/')

for i in range(1,4):
    data = pd.read_excel('17년{}월 LGD구미 투입실적현황_(송부용).xls'.format(i), sheetname = 'raw-data', encoding = 'euc-kr', 
                         skiprows = [0] + [i for i in range(2,2296)], parse_cols = ('B:BQ'))
    data = data[data['Maker'] == 'LGC']
    data.reset_index(drop = True, inplace = True)
    
    data['연신'], data['코팅'] = 'Lot','Lot'
    lot = data['Lot No.']
    
    for i in range(len(lot)):
        findlot = pd.read_sql_query(qs.find_lot(lot.ix[i]), db)
        try : 
            data.set_value(i, '연신', findlot['연신LOT'][0])
            data.set_value(i, '코팅', findlot['코팅LOT'][0])
            print('working')
            
        except : 
            data.set_value(i, '연신', '확인 불가')
            data.set_value(i, '코팅', '확인 불가')
    
    
data['이물계불량'] = data['(TAB)\n내부이물_실오라기'] + data['(TAB)\n내부이물_액정계'] + data['(TAB)\n내부이물_비액정계'] + data['(TAB)\n실오라기.내부']
        
data.to_excel('{}월(구미).xlsx'.format(i), index = False)

In [ ]:
#%% 연태 투입실적 정리
Folder = '//Client/D$/#.Secure Work Folder/2.투입실적/17년 투입 실적/연태'

os.chdir(Folder)

input_data = pd.read_excel('//Client/D$/#.Secure Work Folder/2.투입실적/17년 투입 실적/연태/2017-04-18 Daily Report.xlsx', sheetname = '投入数量', parse_cols = ('B:L'))
#%%
input_data['연신'], input_data['코팅'], input_data['코드'] = 'Lot', 'Lot', '코드'
lot = input_data['LOT NO'] 
#%%
for i in range(len(lot)):
    findlot = pd.read_sql_query(qs.lottracer(qs.lotchanger(lot.ix[i])), db1).drop_duplicates(subset = 'PROD_WC_CD').sort_values(by = 'PROD_WC_CD', ascending = False).reset_index(drop = True)
    try:
        input_data.set_value(i, '연신', findlot['UNIQUE_LOT_NO'][0])
        input_data.set_value(i, '코팅', findlot['UNIQUE_LOT_NO'][1])
        input_data.set_value(i, '코드', findlot['PROD_CD'][1])
        print('working')
    except :
        input_data.set_value(i, '연신', '확인 불가')
        input_data.set_value(i, '코팅', '확인 불가')
        input_data.set_value(i, '코드', '확인 불가')

input_data['Date'] = input_data['Date'].apply(lambda x: pd.Timestamp('1899-12-30') + pd.Timedelta(days = x))

input_data.to_excel('hello.xlsx', index = False)